# Sound Magician

Thời lượng ước tính: **120** phút

## Giới thiệu lab
Nâng cao! Có thể bạn đã từng xem một bộ phim hoặc chương trình truyền hình mà các điệp viên siêu hạng sử dụng một số kỹ thuật khó tin để nâng cao hình ảnh pixelated hoặc khôi phục một số dữ liệu bị mất. Trong lab này, bạn sẽ thực hiện điều tương tự — lần này mục tiêu của bạn là sử dụng hồi quy tuyến tính để khôi phục hoặc 'điền vào' phần bị xóa hoàn toàn của một tệp âm thanh!

Để hoàn thành lab này, bạn sẽ sử dụng FSDD (Free-Spoken-Digits-Dataset), một tập dữ liệu âm thanh do Zohar Jackson tổng hợp lại khi anh ấy nhận thấy không có nhiều âm thanh được làm sạch (không có khoảng trống, độ dài tương đương, cùng bitrate, cùng tỷ lệ mẫu trên giây, ...) thư viện âm thanh sẵn sàng cho machine learning.

In [1]:
import numpy as np
import pandas as pd
import os

from sklearn.utils.validation import check_random_state
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import scipy.io.wavfile as wavfile

### Về Âm thanh

Mẫu là các quan sát. Mỗi tệp âm thanh sẽ là một mẫu duy nhất trong tập dữ liệu.

Tìm thêm thông tin về [Mẫu âm thanh tại đây] (https://en.wikipedia.org/wiki/Sampling_(signal_processing)).

Mỗi tệp .wav thực sự chỉ là một loạt các mẫu số, được lấy mẫu từ tín hiệu analog. Sampling (Lấy mẫu) là một kiểu rời rạc. Khi chúng ta đề cập đến các 'sample (mẫu)', tức là các quan sát. Khi đề cập đến 'audio sample', chúng ta muốn nói đến "feature (thuộc tính)" thực sự của tệp âm thanh.

Mục tiêu của lab này là sử dụng hồi quy tuyến tính, đa mục tiêu để tạo ra bằng phép ngoại suy phần bị thiếu của tệp âm thanh thử nghiệm.

Mỗi một thuộc tính audio_sample sẽ là đầu ra của một phương trình, là một hàm của phần được cung cấp của audio_samples:

    missing_samples = f(provided_samples)

Bạn có thể thử với lượng âm thanh bạn muốn cắt và để máy tính tạo ra bằng tham số Provided_Portion.

Hãy sử dụng cái này. Đây là lượng tệp âm thanh sẽ được cung cấp, tính bằng phần trăm. Phần trăm còn lại của tệp sẽ được tạo thông qua phép ngoại suy tuyến tính.

In [2]:
Provided_Portion = 0.25

## 1. Chuẩn bị dữ liệu

Bắt đầu bằng cách tạo một list Python thông thường là `zero`:

In [3]:
audio_sample = []
print(audio_sample)

[]


Lặp qua tập dữ liệu và tải lên tất cả 50 tệp `0_jackson*.wav` bằng phương thức `wavfile.read()`: https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.io.wavfile.read.html từ thư mục [data](https://drive.google.com/drive/folders/1UDAvuRfXzcOcpI84neQAmAeffIrJKKBi?usp=sharing).

Hãy cẩn thận! `.read()` trả về một tuple và bạn chỉ quan tâm đến dữ liệu âm thanh chứ không phải sample_rate tại thời điểm này. Bên trong vòng lặp for, chỉ cần nối dữ liệu âm thanh đã tải vào list Python `zero`:

In [4]:
data_directory = 'data'
for i in range(50):
    file_path = os.path.join(data_directory, f'0_jackson_{i}.wav')
    sample_rate, data = wavfile.read(file_path)
    audio_sample.append(data)
        
print(audio_sample)

[array([-369, -431, -475, ...,  301,  324,  304], dtype=int16), array([-311,  -91, -140, ...,  378,  357,  333], dtype=int16), array([-361, -226, -238, ..., -286, -311, -343], dtype=int16), array([ 323,  338,  357, ..., -246, -280, -301], dtype=int16), array([-302, -312, -103, ..., -338, -333, -348], dtype=int16), array([ 305,  365,  419, ..., -313, -342, -346], dtype=int16), array([ 364,  420,  469, ..., -326, -334, -351], dtype=int16), array([-417,  152,  168, ...,  312,  316,  309], dtype=int16), array([ 330,  382,  389, ..., -410, -400, -385], dtype=int16), array([-312, -335, -338, ...,  384,  371,  345], dtype=int16), array([-314, -303, -332, ..., -355, -343, -322], dtype=int16), array([347, 351, 462, ..., 365, 338, 302], dtype=int16), array([-336,  160,   65, ..., -315, -343, -319], dtype=int16), array([ 354,  442,  610, ..., -312, -336, -333], dtype=int16), array([ 397,  531,  638, ..., -357, -386, -353], dtype=int16), array([ 382,  459,  530, ..., -254, -301, -309], dtype=int16

Hãy dành một chút thời gian để chuyển 0 thành một khung dữ liệu (DataFrame). Khi bạn làm như vậy, hãy đặt `dtype` thành `np.int16`, vì các tệp âm thanh đầu vào là 16 bit cho mỗi mẫu. Nếu bạn không biết cách thực hiện việc này, hãy đọc tài liệu tại đây: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html

Tiếc là những đoạn âm thanh này không được chuẩn hóa độ dài nên chúng ta sẽ phải chia nhỏ chúng để tất cả có cùng độ dài. Vì Pandas sẽ chèn các NAN tại bất kỳ vị trí nào để tạo thành mảng [n_observed_samples, n_audio_samples] hình chữ nhật hoàn hảo nên hãy thực hiện một `dropna` trên trục Y ở đây. Sau đó, chuyển đổi 1 trở lại thành NDArray bằng `yourarrayname.values`:

In [5]:
# Tìm độ dài ngắn nhất trong tất cả các đoạn âm thanh
min_length = min(len(sample) for sample in audio_sample)

# Cắt các đoạn âm thanh để tất cả có cùng độ dài ngắn nhất
audio_sample = [sample[:min_length] for sample in audio_sample]

# Tạo DataFrame từ danh sách các đoạn âm thanh đã chuẩn hóa
audio_df = pd.DataFrame(audio_sample, dtype=np.int16)

# Thực hiện dropna để loại bỏ bất kỳ hàng nào chứa giá trị NaN (nếu có)
audio_df.dropna(axis=0, inplace=True)

# Chuyển đổi DataFrame trở lại NDArray
audio_array = audio_df.values

# Kiểm tra NDArray đã được tạo
print(audio_array)

[[-369 -431 -475 ... -108 -402 -884]
 [-311  -91 -140 ... -422 -462 -460]
 [-361 -226 -238 ...  378  416  438]
 ...
 [ 320  372  421 ... 1433 1616 1427]
 [-439 -572 -656 ... 1026 2322 2618]
 [-302 -337 -371 ... 1105  559  477]]


Điều quan trọng là phải biết  (bao nhiêu mẫu audio_samples) độ dài dữ liệu hiện có.

`zero` hiện có shape giống như` [n_samples, n_audio_samples] `, vì vậy hãy lấy số lượng` n_audio_samples` và lưu trữ nó trong biến `n_audio_samples`:

In [6]:
# Lấy số lượng n_audio_samples từ hình dạng của NDArray
n_audio_samples = audio_array.shape[1]

# Kiểm tra NDArray và n_audio_samples
print("NDArray shape:", audio_array.shape)
print("Number of audio samples (n_audio_samples):", n_audio_samples)

NDArray shape: (50, 4087)
Number of audio samples (n_audio_samples): 4087


## 2. Huấn luyện mô hình

Tạo mô hình hồi quy tuyến tính của bạn tại đây và lưu trữ nó trong biến `model`. Đừng vội huấn luyện hoặc làm bất cứ điều gì khác với nó:

In [7]:
model = LinearRegression()

Có 50 bản ghi mỗi clip. Bạn chỉ muốn lấy ngẫu nhiên một trong số chúng ra và cái đó sẽ KHÔNG được sử dụng trong quá trình huấn luyện mô hình. Nói cách khác, tệp mà chúng ta sẽ kiểm tra/chấm điểm sẽ là một mẫu không nhìn thấy, độc lập với phần còn lại của training set:

In [ ]:
# Hãy để nguyên dòng này cho đến khi bạn đã submit bài Lab của mình:
rng = check_random_state(7)

random_idx = rng.randint(zero.shape[0])
test  = zero[random_idx]
train = np.delete(zero, [random_idx], axis=0)

In ra shape của `train` và `test`.

`train` sẽ có dạng:`[n_samples, n_audio_samples]`, trong đó `n_audio_samples` là 'features' (thuộc tính) của tệp âm thanh

`test` sẽ có dạng `[n_audio_features]`, vì nó là mẫu duy nhất (tệp âm thanh, ví dụ: quan sát).

In [10]:
# Chọn ngẫu nhiên một mẫu để làm mẫu kiểm tra
rng = check_random_state(7)
random_idx = rng.randint(audio_array.shape[0])
test_sample = audio_array[random_idx]
train_samples = np.delete(audio_array, [random_idx], axis=0)

print("Train NDArray shape:", train_samples.shape)
print("Test sample shape:", test_sample.shape)

Train NDArray shape: (49, 4087)
Test sample shape: (4087,)


Dữ liệu thử nghiệm sẽ có hai phần, `X_test` và` y_test`.

`X_test` là phần đầu tiên của tệp âm thanh thử nghiệm mà chúng ta sẽ cung cấp cho máy tính làm input.

`y_test`,"nhãn" sẽ là phần còn lại của tệp âm thanh. Như vậy, máy tính sẽ sử dụng hồi quy tuyến tính để lấy ra phần bị thiếu của tệp âm thanh dựa trên dữ liệu huấn luyện mà nó nhận được!

Hãy lưu clip `test` ban đầu, clip mà bạn sắp xóa một nửa vào thư mục hiện tại để bạn có thể so sánh nó với clip 'đã vá' sau khi tạo. Bạn hẳn đã có `sample_rate` khi tải lên các tệp .wav:

In [11]:
# Lưu clip âm thanh kiểm tra ban đầu
wavfile.write('Original_Test_Clip.wav', sample_rate, test_sample)

# Chia test_sample thành X_test và y_test
split_index = len(test_sample) // 2
X_test = test_sample[:split_index]
y_test = test_sample[split_index:]

Chuẩn bị dữ liệu TEST bằng cách tạo lát `X_test`. Nó phải có các thuộc tính mẫu âm thanh `Provided_Portion` * `n_audio_samples`, được lấy từ tệp âm thanh thử nghiệm, hiện được lưu trữ trong biến `test`. Nói cách khác, lấy các thuộc tính âm thanh `Provided_Portion` * `n_audio_samples` ĐẦU TIÊN từ `test` và lưu trữ nó trong` X_test`. Điều này sẽ được thực hiện bằng cách sử dụng lập chỉ mục:

In [12]:
# Số lượng thuộc tính âm thanh mà chúng ta sẽ lấy từ test_sample
num_audio_features = int(Provided_Portion * n_audio_samples)

# Tạo X_test bằng cách lấy các thuộc tính âm thanh đầu tiên từ test_sample
X_test = test_sample[:num_audio_features]

# In ra shape của X_test
print("X_test shape:", X_test.shape)

X_test shape: (1021,)


Nếu các thuộc tính `Provided_Portion` * `n_audio_samples` đầu tiên được lưu trữ trong `X_test`, thì chúng ta cũng cần lấy các thuộc tính âm thanh _còn lại_ và lưu trữ chúng trong` y_test`. Với các thuộc tính còn lại được lưu trữ trong đó, chúng ta sẽ có thể R ^ 2 "chấm điểm" thuật toán đã làm tốt như thế nào trong việc hoàn thành tệp âm thanh.

In [13]:
# Các thuộc tính âm thanh còn lại là phần từ vị trí num_audio_features trở đi
y_test = test_sample[num_audio_features:]

# In ra shape của y_test
print("y_test shape:", y_test.shape)

y_test shape: (3066,)


Lặp lại quy trình tương tự cho `X_train`, `y_train`. Sự khác biệt duy nhất là:

1. Bạn sẽ nhận được dữ liệu âm thanh từ `train` thay vì từ `test`
2. Bạn có nhớ shape của `train` mà bạn đã in ra trước đó không? Bạn muốn thực hiện việc cắt này nhưng đối với TẤT CẢ các mẫu (quan sát). Đối với mỗ quan sát, bạn muốn chia các thuộc tính âm thanh `Provided_Portion` * `n_audio_samples` đầu tiên thành `X_train`, và phần còn lại chuyển vào `y_train`. Có thể thực hiện tất cả những điều này bằng lập chỉ mục thông thường ở 2 dòng code:

In [14]:
# Số lượng thuộc tính âm thanh mà chúng ta sẽ lấy từ mỗi mẫu train
num_audio_features = int(Provided_Portion * n_audio_samples)

# Tạo X_train và y_train từ tất cả các mẫu trong train_samples
X_train = train_samples[:, :num_audio_features]
y_train = train_samples[:, num_audio_features:]

# In ra shape của X_train và y_train
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (49, 1021)
y_train shape: (49, 3066)


SciKit-Learn sẽ 'tức giận' nếu bạn không cung cấp dữ liệu huấn luyện dưới dạng khung dữ liệu 2D có shape `[n_samples, n_features]`.

Vì vậy, nếu bạn chỉ có một MẪU, chẳng hạn như trường hợp của chúng ta với `X_test` và `y_test` thì bằng cách gọi `.reshape(1, -1)`, bạn có thể biến `[n_features]` thành `[1, n_features] `để đáp ứng SciKit-Learn.

Mặt khác, nếu bạn chỉ có một THUỘC TÍNH, bạn có thể gọi `.reshape (-1, 1)` trên dữ liệu của mình để biến `[n_samples]` thành `[n_samples, 1]`.

Định hình lại X_test và y_test thành [1, n_features]:

In [15]:
# Định hình lại X_test và y_test
X_test = X_test.reshape(1, -1)
y_test = y_test.reshape(1, -1)

# In ra shape của X_test và y_test đã định hình lại
print("X_test reshaped shape:", X_test.shape)
print("y_test reshaped shape:", y_test.shape)

X_test reshaped shape: (1, 1021)
y_test reshaped shape: (1, 3066)


Điều chỉnh mô hình của bạn bằng cách sử dụng dữ liệu huấn luyện và nhãn:

In [16]:
# Huấn luyện mô hình trên dữ liệu huấn luyện
model.fit(X_train, y_train)

# Sau khi huấn luyện, bạn có thể kiểm tra các thông số của mô hình, ví dụ:
print("Intercept:", model.intercept_)
print("Coefficients:", model.coef_)

Intercept: [-392.99658585 -439.93269982 -193.62765415 ...  228.0534521   -13.5785882
 -345.71072916]
Coefficients: [[-0.01006156 -0.01222838 -0.01431177 ...  0.07957438  0.11474667
   0.13375861]
 [-0.00747665 -0.0087656  -0.01020086 ...  0.05056315  0.09132977
   0.1148416 ]
 [-0.00249175 -0.00396567 -0.00590694 ... -0.00725308  0.03584432
   0.11652158]
 ...
 [ 0.00865076  0.01170158  0.01206544 ...  0.02779679  0.0312181
   0.04062864]
 [ 0.0016898   0.00418021  0.00360833 ...  0.01723349  0.02001436
   0.0316697 ]
 [-0.00752646 -0.00656857 -0.00795753 ...  0.00957986  0.0106107
   0.01602369]]


## 3. Đánh giá mô hình

Sử dụng mô hình để dự đoán `label` của `X_test`. Lưu trữ dự đoán kết quả trong biến `y_test_prediction`:

In [17]:
# Sử dụng mô hình để dự đoán nhãn của X_test
y_test_prediction = model.predict(X_test)

# In ra shape của y_test_prediction để kiểm tra
print("y_test_prediction shape:", y_test_prediction.shape)

y_test_prediction shape: (1, 3066)


SciKit-Learn sẽ sử dụng float64 để tạo các dự đoán của bạn, vì vậy hãy đưa các giá trị đó trở lại int16, đó là những gì tệp .wav cần:

In [18]:
y_test_prediction = y_test_prediction.astype(dtype=np.int16)

Chấm điểm dự đoán của bạn sẽ hoạt động tốt như thế nào bằng cách truyền dữ liệu và nhãn thử nghiệm `y_test`:

In [19]:
# Tính điểm R^2 cho mô hình
score = model.score(X_test, y_test)

C:\Users\PC\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [20]:
# Tính Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_test_prediction)

# Tính Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_test_prediction)

In [21]:
# In ra MSE và MAE
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)

Mean Squared Error (MSE): 696.544357469015
Mean Absolute Error (MAE): 6201.038160469668


Hãy lấy phần `Provided_Portion` đầu tiên của clip thử nghiệm, phần mà bạn đã đưa vào mô hình hồi quy tuyến tính. Sau đó, kết hợp điều đó với thứ mà mô hình dự đoán đã tạo cho bạn và sau đó lưu đoạn âm thanh đã hoàn thành:

In [22]:
completed_clip = np.hstack((X_test, y_test_prediction))
wavfile.write('Extrapolated Clip.wav', sample_rate, completed_clip[0])